In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns


from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np


In [2]:
df = pd.read_csv ("D:\\Python basics\\CLTV-360\\CSV files\\rfm_segments.csv")
df.head(1)

,CustomerID,Recency,Customer_Age,Frequency,Monetary,ChurnProbab,AOV,PurchaseFrequency,Expected_Lifespan,CLTV,Segment
0,12346.0,326,0,1,77183.6,1.0,77183.6,1.0,0.003,115.78,3


In [3]:
df.drop(['CustomerID', 'Segment'], axis=1, inplace=True)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df.drop('CLTV', axis=1))
y = df['CLTV']


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [6]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)


In [7]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor()
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_test)


In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

def evaluate(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{model_name} - RMSE: {rmse:.2f}, MAE: {mae:.2f}")

evaluate(y_test, y_pred_lr, "Linear Regression")
evaluate(y_test, y_pred_xgb, "XGBoost")


Linear Regression - RMSE: 1168.13, MAE: 196.92
XGBoost - RMSE: 1597.44, MAE: 74.46


In [9]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}


In [10]:
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print(" Best Parameters:", grid_search.best_params_)


Fitting 3 folds for each of 72 candidates, totalling 216 fits
 Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.8}


In [11]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f" Tuned XGBoost RMSE: {rmse:.2f}, MAE: {mae:.2f}")


 Tuned XGBoost RMSE: 954.15, MAE: 47.49


In [12]:
import joblib
joblib.dump(best_model, 'xgb_cltv_model.pkl')


['xgb_cltv_model.pkl']